In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/travel-insurance-prediction-data/TravelInsurancePrediction.csv


In [41]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer

In [9]:
sns.set_theme(context='talk', style='darkgrid', font='MS GOTHIC')
df = pd.read_csv('/kaggle/input/travel-insurance-prediction-data/TravelInsurancePrediction.csv', index_col=0)
df.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,31,Government Sector,Yes,400000,6,1,No,No,0
1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,34,Private Sector/Self Employed,Yes,500000,4,1,No,No,1
3,28,Private Sector/Self Employed,Yes,700000,3,1,No,No,0
4,28,Private Sector/Self Employed,Yes,700000,8,1,Yes,No,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 0 to 1986
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Age                  1987 non-null   int64 
 1   Employment Type      1987 non-null   object
 2   GraduateOrNot        1987 non-null   object
 3   AnnualIncome         1987 non-null   int64 
 4   FamilyMembers        1987 non-null   int64 
 5   ChronicDiseases      1987 non-null   int64 
 6   FrequentFlyer        1987 non-null   object
 7   EverTravelledAbroad  1987 non-null   object
 8   TravelInsurance      1987 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 155.2+ KB


In [11]:
# 各カラムのnuniqueとunique値
for col in df.columns:
    print(f'uniques of {col} is ( {df[col].nunique()} ) : \n {df[col].unique()}')
    print('===================================')

uniques of Age is ( 11 ) : 
 [31 34 28 25 33 26 32 29 35 30 27]
uniques of Employment Type is ( 2 ) : 
 ['Government Sector' 'Private Sector/Self Employed']
uniques of GraduateOrNot is ( 2 ) : 
 ['Yes' 'No']
uniques of AnnualIncome is ( 30 ) : 
 [ 400000 1250000  500000  700000 1150000 1300000 1350000 1450000  800000
 1400000  850000 1500000 1050000  350000 1100000  600000  900000  550000
  300000  750000 1200000 1000000  950000 1700000 1750000  650000  450000
 1650000 1800000 1550000]
uniques of FamilyMembers is ( 8 ) : 
 [6 7 4 3 8 9 5 2]
uniques of ChronicDiseases is ( 2 ) : 
 [1 0]
uniques of FrequentFlyer is ( 2 ) : 
 ['No' 'Yes']
uniques of EverTravelledAbroad is ( 2 ) : 
 ['No' 'Yes']
uniques of TravelInsurance is ( 2 ) : 
 [0 1]


In [13]:
df.columns

Index(['Age', 'Employment Type', 'GraduateOrNot', 'AnnualIncome',
       'FamilyMembers', 'ChronicDiseases', 'FrequentFlyer',
       'EverTravelledAbroad', 'TravelInsurance'],
      dtype='object')

In [28]:
cat_features = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']
num_features = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']

X = df.drop(columns='TravelInsurance')
y = df['TravelInsurance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17, stratify=y)

scaler = StandardScaler()
encoder = OneHotEncoder(drop='first')

ct = ColumnTransformer(
    [
        ('num_process', scaler, num_features),
        ('cat_process', encoder, cat_features)
    ],
    remainder='passthrough'
)

X_train_transformed = ct.fit_transform(X_train)
X_test_transformed = ct.transform(X_test)
X_train_transformed[0, :]

array([ 1.13454572, -0.37974587,  0.76972512, -0.61493412,  0.        ,
        1.        ,  0.        ,  0.        ])

In [30]:
X_train_transformed[:, 0].std()

1.0

In [35]:
kn_model = KNeighborsClassifier()
sv_model = SVC(random_state=17)

In [36]:
kn_model.fit(X_train_transformed, y_train)
sv_model.fit(X_train_transformed, y_train)

SVC(random_state=17)

In [37]:
print(kn_model.score(X_test_transformed, y_test))
print(sv_model.score(X_test_transformed, y_test))

0.7738693467336684
0.8040201005025126


In [50]:
models = [
          {'model': kn_model, 'params':{'n_neighbors':[10, 15, 20, 25], 'weights':['uniform', 'distance']}}, 
          {'model': sv_model, 'params':{'C':[0.1, 1, 10, 20], 'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['auto', 'scale']}}
         ]
k = StratifiedKFold(5, shuffle=True, random_state=17)

In [51]:
dfs = []
for model in models:
    grid = GridSearchCV(
        model['model'],
        param_grid=model['params'],
        cv=k
    )
    
    grid.fit(X_train_transformed, y_train)
    df = pd.DataFrame(grid.cv_results_).loc[:, 'params':].sort_values('rank_test_score', ascending=True)[:5]
    dfs.append(df)
    print(f"best params of {model['model']} : {grid.best_params_}\n")

best params of KNeighborsClassifier() : {'n_neighbors': 15, 'weights': 'uniform'}

best params of SVC(random_state=17) : {'C': 20, 'gamma': 'scale', 'kernel': 'poly'}



In [52]:
kn_best_model = KNeighborsClassifier(n_neighbors=25, weights='uniform')
sv_best_model = SVC(C=1, gamma='scale', kernel='poly')

In [53]:
print(kn_model.score(X_train_transformed, y_train))
print(sv_model.score(X_train_transformed, y_train))

0.8332284455632474
0.830081812460667
